## Information:
Unfortunately, JupyterLab does not support displaying logs/status progress while a cell is running. Instead, everything is displayed at the end. Colab, on the other hand, displays the logs/status progress correctly.

In [ ]:
! git clone https://github.com/RobinMaas95/GTSRB_Visualization.git

In [ ]:
if 'google.colab' in str(get_ipython()):
    colab=True
else:
    colab=False

## Setup python file paths

In [ ]:
import sys
from pathlib import Path

# Define paths
base_path = Path.cwd()

## If we use a local machine, we are presumably in the code folder. If so, change to parent folder
if base_path.parent.name == "GTSRB_Visualization":
    base_path = base_path.parent
    %cd ..
    
colab_suffix = "GTSRB_Visualization" if colab else ""   

code_path = base_path.joinpath(colab_suffix, "code")
test_performance_path = code_path.joinpath("test_performance.py")



In [ ]:
# Only neccessary if you working in colab. If you use your local system, please use the requirements.txt
if colab:
  ! pip -q install pytorch-lightning --upgrade
  sys.path.insert(0, "/content/GTSRB_Visualization/code")


In [ ]:
# Now all imports are possible
import warnings
import zipfile
from pathlib import Path
from shutil import copyfile

import pandas as pd
import seaborn as sns
import torch
from google import colab
from matplotlib import pyplot as plt
from matplotlib import ticker as ticker
from model import LitModel
from PIL import Image
from skimage import io
from torchvision import transforms as transforms

warnings.filterwarnings("ignore")

# Copy files (COLAB)

In [ ]:
if colab:
    ! cat /content/GTSRB_Visualization/checkpoints/splitted_trained_model_a* > /content/trained_model.zip
    ! unzip -qq /content/trained_model.zip
    ! cp "/content/GTSRB_Visualization/checkpoints/trained_model_org.ckpt" /content/trained_model_org.ckpt
    trained_model_path = "/content/trained_model.ckpt"
    trained_model_org_path = "/content/trained_model_org.ckpt"
else:
    print("You are not running on colab, use next section to set your paths!")

In [ ]:
if colab:
    ! cp -r /content/GTSRB_Visualization/data/cropped_test /content/cropped_test
    ! cp -r /content/GTSRB_Visualization/data/original_test /content/original_test
    ! cp -r /content/GTSRB_Visualization/data/masking_results /content/masking_results
    ! cp -r /content/GTSRB_Visualization/data/real_examples /content/real_examples
    ! cp -r /content/GTSRB_Visualization/data/sticker /content/sticker
    cropped_test_path = Path("/content/cropped_test")
    original_test_path = Path("/content/original_test")
    masking_results_path = Path("/content/masking_results")
    real_examples_path = Path("/content/real_examples")
    sticker_path = Path("/content/sticker")
else:
    print("You are not running on colab, use next section to set your paths!")

## Set Paths (LOCAL ENV)
Setting the paths to the individual files or folders

In [ ]:
if not colab:
    trained_model_path = Path("") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/checkpoints/trained_model.ckpt"
    trained_model_org_path = Path("") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/checkpoints/trained_model_org.ckpt"
    cropped_test_path = Path("") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/data/cropped_test/"
    original_test_path = Path("") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/data/original_test/"
    masking_results_path = Path(r"C:/Users/Robin/Desktop/GTSRB_Visualization/data/masking_results/") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/data/masking_results/"
    real_examples_path = Path("") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/data/real_examples/"
    sticker_path = Path("") # E.g. "C:/Users/Robin/Desktop/GTSRB_Visualization/data/sticker/"
else:
    print("You are running on colab, use the section above!")

You are running on colab, use the section above!


## Set rotated/perspective path (after you created them by yourself)

In [ ]:
# Due to the limited size of a repository on Github, these datasets cannot be provided. Please use the file "rotate_dataset.ipynb" and "perspective/main.py" to create these folders yourself and then adjust the paths here.
perspective_path = "" # E.g. # E.g. "C:\Users\Robin\Desktop\GTSRB_Visualization\data\rotated\"
rotated_path = "" # E.g. "C:\Users\Robin\Desktop\GTSRB_Visualization\data\perspective\"

## Generate Folderpaths

In [ ]:
grad_cam_path = masking_results_path.joinpath("heatmap_grad_cam", "masked_heatmap_grad_cam_")
grad_cam_pp_path = masking_results_path.joinpath("heatmap_grad_cam_pp", "masked_heatmap_grad_cam_pp_")
saliency_path = masking_results_path.joinpath("heatmap_saliency", "masked_heatmap_saliency_")
sticker_org_path = sticker_path.joinpath("original")
sticker_f_large_path = sticker_path.joinpath("fortuna", "large_sign")
sticker_f_medium_path = sticker_path.joinpath("fortuna", "medium_sign")
sticker_f_small_path = sticker_path.joinpath("fortuna", "small_sign")
sticker_b_large_path = sticker_path.joinpath("bochum", "large_sign")
sticker_b_medium_path = sticker_path.joinpath("bochum", "medium_sign")
sticker_b_small_path = sticker_path.joinpath("bochum", "small_sign")



## Generell function

In [ ]:
FILETYPE_LIST = ["pdf", "png", "svg"]


def create_clean_df(path, acc_name_tuple, to_be_index_name="bla"):
    df = pd.read_csv(path, index_col=0)
    df.sort_values(to_be_index_name, inplace=True)
    df.set_index(to_be_index_name, inplace=True)
    df.rename(columns={acc_name_tuple[0]: acc_name_tuple[1]}, inplace=True)
    df.index.name = None

    return df


def save_images(filename, fig):
    for file_type in FILETYPE_LIST:
        fig.savefig(f"{filename}.{file_type}")

# Run Performance Tests

## Set mean/std variables
Use the correct values for your dataset, if you created one!
Set mean/std variables. Use the correct values for your dataset, if you created one!

You find them in the logs under the build_model.py call, e.g.:


```
/content/GTSRB_Visualization/code/build_model.py - INFO - Mean: [0.32352597020361307, 0.2917746800988572, 0.31248061517888454]
/content/GTSRB_Visualization/code/build_model.py - INFO - Std: [0.2770588879067902, 0.2602109151608635, 0.2735251700292777]
```

Replace the values of the _list variables with the lists at the end of the log lines:

 

In [ ]:
mean_list = [0.3232291265307408, 0.2916548292210219, 0.31239060828717274]
std_list = [0.2765753268041087, 0.26007308085671965, 0.27338755423173944]
mean = f"{mean_list[0]} {mean_list[1]} {mean_list[2]}"
std = f"{std_list[0]} {std_list[1]} {std_list[2]}"

## Original Datasets

In [ ]:
# Passing the specific mean/std values from the original dataset, later we 
# will use the mean/std variables for this, but they contain the values
# for the cropped datasets
! python /content/GTSRB_Visualization/code/test_performance.py --checkpoint $trained_model_org_path --method_names "Original"  \
   --datasets $trained_model_org_path \
   --mean 0.31191239122335596 0.2876613986697896 0.29806319801822573 \
   --std 0.2787123625790472 0.26702313360428914 0.2753398226529556

In [ ]:
# No need to pass mean/std because of the default values in test_performance.py
# If you created your own datasets, make sure that you pass the right mean/std values
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Original Cropped"  \
  --datasets $cropped_test_path \
  --mean $mean \
  --std $std

## Masked Datasets

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "GradCam" "GradCam++" "Saliency Map" \
  --datasets $grad_cam_path $grad_cam_pp_path $masked_headmap_path \
  --mean $mean \
  --std $std

## Rotated/Perspective



In [ ]:
rotated_folder_path = rotated_path.joinpath("rotated_test_crop_degree_")

! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Rotated" \
  --datasets $rotated_folder_path \
  --mean $mean \
  --std $std

In [ ]:
perspective_folder_path = perspective_path.joinpath("perspective_degree_")

! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Perspective" \
  --datasets perspective_folder_path \
  --mean $mean \
  --std $std

# Count files in folder

In [ ]:
def get_num_files_in_folder(folder_dict):
    result_dict = {}
    for key, value in folder_dict.items():
        p = Path(value).glob("**/*")
        files = [x for x in p if x.is_file()]
        result_dict[key] = len(files)

    return result_dict

In [ ]:
folder_dict = {
    "GradCam": str(grad_cam_path) + "01",
    "Saliency Map": str(saliency_path) + "01",
    "GradCam++": str(grad_cam_pp_path) + "01",
    "Original": cropped_test_path,
}

num_files_dict = get_num_files_in_folder(folder_dict)

In [ ]:
fig = plt.figure(figsize=(10, 8))
df = pd.DataFrame(num_files_dict.items(), columns=["Method", "Number of Files"])
splot = sns.barplot(
    x="Method",
    y="Number of Files",
    data=df.sort_values(by="Number of Files", ascending=False),
)
for p in splot.patches:
    splot.annotate(
        format(p.get_height(), ".0f"),
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        xytext=(0, 9),
        textcoords="offset points",
    )
plt.ylabel("Anzahl der Bilder", size=14)
plt.xlabel("Visualisierungs Methode", size=14)

In [ ]:
name = "overview_number_files_methods"
save_images(name, fig)
df.to_csv(name, index=False)

# Plot results

# Perspective

In [ ]:
df = pd.read_csv(str(base_path.joinpath("perspective_degree__df.csv")))

sns.set_theme(style="whitegrid")
seaborn_plot = sns.relplot(data=df, x="Perspective", y="Test accuracy")
plt.ylabel("Test Accuracy")
plt.xlabel("Grad Rotation Z-Achse")

In [ ]:
# Center around 0 Degree
df["Perspective"] = df["Perspective"].apply(lambda x: x if x < 100 else x - 360)
df.to_csv(str(base_path.joinpath("perspective_degree__df.csv")), index=False)

# Clean up df
df = create_clean_df(
    str(base_path.joinpath("perspective_degree__df.csv")),
    ("Test accuracy", "Test Accuracy"),
    to_be_index_name="Perspective",
)

In [ ]:
sns.set_theme(style="whitegrid")
seaborn_plot = sns.relplot(data=df, x=df.index, y="Test Accuracy")
plt.ylabel("Test Accuracy")
plt.xlabel("Grad Rotation Z-Achse")

In [ ]:
name = "perspective_degree"
save_images(name, seaborn_plot)
df.to_csv(name + ".csv", index=False)

## Rotated

In [ ]:
df = pd.read_csv(str(base_path.joinpath("rotated_test_crop_degree__df.csv")))

sns.set_theme(style="whitegrid")
seaborn_plot = sns.relplot(data=df, x="Rotated", y="Test accuracy")
plt.ylabel("Test Accuracy")
plt.xlabel("Grad Rotation X-Achse")

In [ ]:
# Center around 0 Degree
df["Rotated"] = df["Rotated"].apply(lambda x: x if x < 100 else x - 360)
df.to_csv(str(base_path.joinpath("rotated_test_crop_degree__df.csv")), index=False)

# Clean up df
df = create_clean_df(
    str(base_path.joinpath("rotated_test_crop_degree__df.csv")),
    ("Test accuracy", "Test Accuracy"),
    to_be_index_name="Rotated",
)
df.to_csv(str(base_path.joinpath("rotated_degree.csv")), index=False)

In [ ]:
sns.set_theme(style="whitegrid")
seaborn_plot = sns.relplot(data=df, x=df.index, y="Test Accuracy")
plt.ylabel("Test Accuracy")
plt.xlabel("Grad Rotation X-Achse")

In [ ]:
name = "rotated_degree"
save_images(name, seaborn_plot)
df.to_csv(name + ".csv", index=False)

## Create GradCam Folder equal to GradCam++
GradCam++ has less heat maps with activation than GradCam. Since the masking was only applied to heatmaps with activation, the performance evaluation is performed on fewer images. If these are "worse" images, this could cause GradCam++ to degrade more. To compensate for this, a dataset is created which contains the images from GradCam, which are also available in GradCam++.

In [ ]:
grad_cam_pp_folder = str(grad_cam_pp_path) + "01"
grad_cam_folder = grad_cam_path
grad_cam_cutted = masking_results_path.joinpath("heatmap_grad_cam_cutted", "masked_heatmap_grad_cam_cutted_")

p = Path(grad_cam_pp_folder).glob("**/*")
files = [x for x in p if x.is_file()]

In [ ]:
for i in range(1, 11):
    for cfile in files:
        src = str(
            Path(str(grad_cam_folder) + str(i).zfill(2)).joinpath(
                cfile.parent.name, cfile.name
            )
        )
        dest = Path(str(grad_cam_cutted) + str(i).zfill(2)).joinpath(
            cfile.parent.name, cfile.name
        )
        dest.parent.mkdir(parents=True, exist_ok=True)
        try:
            copyfile(src, str(dest))
        except FileNotFoundError:
            # For some images there might be a GradCam++ heatmap, but no GradCam heatmap
            pass

In [ ]:
folder_dict = {
    "GradCam++": str(grad_cam_pp_path) + "01",
    "GradCam Cutted": str(grad_cam_cutted) + "01",
}

results = get_num_files_in_folder(folder_dict)
print(results)
print(
    "--> "
    + str(results.get("GradCam++") - results.get("GradCam Cutted"))
    + " Images less in GradCam"
)

{'GradCam++': 10426, 'GradCam Cutted': 10206}
--> 220 Images less in GradCam


In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_name "GradCam Cutted" \
  --datasets $grad_cam_cutted \
  --mean $mean \
  --std $std

# Compare Results Masking

## Prepare result df

In [ ]:
df_org = create_clean_df(
    base_path.joinpath("cropped_test_df.csv"), ("Test accuracy", "Original"), "Original Cropped"
)
df_sm = create_clean_df(
    base_path.joinpath("masked_heatmap_saliency__df.csv"),
    ("Test accuracy", "Saliency Map"),
    "Saliency Map",
)
df_gradcam = create_clean_df(
    base_path.joinpath("masked_heatmap_grad_cam__df.csv"), ("Test accuracy", "GradCam"), "GradCam"
)
df_gradcam_cutted = create_clean_df(
    base_path.joinpath("masked_heatmap_grad_cam_cutted__df.csv"),
    ("Test accuracy", "GradCam Cutted"),
    "GradCam Cutted",
)
df_gcpp = create_clean_df(
    base_path.joinpath("masked_heatmap_grad_cam_pp__df.csv"),
    ("Test accuracy", "GradCam++"),
    "GradCam++",
)

In [ ]:
# Saliency Map and GradCamCutted have no value for no masking, I fill that value with the result from the original dataset
df_sm.loc[0] = df_org["Original"][0]
df_gradcam_cutted.loc[0] = df_org["Original"][0]

In [ ]:
result = pd.concat([df_sm, df_gradcam, df_gradcam_cutted, df_gcpp], axis=1, sort=False)
result

In [ ]:
result.to_csv(str(base_path.joinpath("result_methods.csv")), index=False)

## Plots

In [ ]:
sns.set_theme(style="whitegrid")
splot = sns.lineplot(data=result, markers=True)
splot.set(xlabel="Anzahl maskierte Zellen", ylabel="Test Accuracy")
splot.xaxis.set_major_locator(ticker.MultipleLocator(1))
splot.xaxis.set_major_formatter(ticker.ScalarFormatter())
splot.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
splot.yaxis.set_major_formatter(ticker.ScalarFormatter())
splot.figure.savefig(f"overview_methods_combined_lines.png")
splot.figure.savefig(f"overview_methods_combined_lines.svg")
splot.figure.savefig(f"overview_methods_combined_lines.pdf")

In [ ]:
sns.set_theme(style="whitegrid")
splot = sns.scatterplot(data=result, markers=True)
splot.set(xlabel="Anzahl maskierte Zellen", ylabel="Test Accuracy")
splot.xaxis.set_major_locator(ticker.MultipleLocator(1))
splot.xaxis.set_major_formatter(ticker.ScalarFormatter())
splot.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
splot.yaxis.set_major_formatter(ticker.ScalarFormatter())
splot.figure.savefig(f"overview_methods_combined_scatter.png")
splot.figure.savefig(f"overview_methods_combined_scatter.svg")
splot.figure.savefig(f"overview_methods_combined_scatter.pdf")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True, sharey=True)
sns.lineplot(
    x=df_sm.index,
    y="Saliency Map",
    data=df_sm,
    ax=axes[0, 0],
    color=sns.color_palette()[0],
)
sns.lineplot(
    y="GradCam",
    x=df_gradcam.index,
    data=df_gradcam,
    ax=axes[0, 1],
    color=sns.color_palette()[1],
)
sns.lineplot(
    y="GradCam Cutted",
    x=df_gradcam_cutted.index,
    data=df_gradcam_cutted,
    ax=axes[1, 0],
    color=sns.color_palette()[2],
)
sns.lineplot(
    y="GradCam++",
    x=df_gcpp.index,
    data=df_gcpp,
    ax=axes[1, 1],
    color=sns.color_palette()[3],
)
axes[0, 0].set_title("Saliency Map")
axes[0, 1].set_title("GradCam")
axes[1, 0].set_title("GradCam Cutted")
axes[1, 1].set_title("GradCam++")
#  Make common axis labels
axes[-1, 0].set_ylabel(".", color=(0, 0, 0, 0))
axes[-0, 0].set_ylabel(".", color=(0, 0, 0, 0))
fig.text(0.5, 0.04, "Test Accuracy", va="center", ha="center")
fig.text(
    0.04, 0.5, "Anzahl maskierte Zellen", va="center", ha="center", rotation="vertical"
)

fig.savefig(f"overview_methods_line_grid.png")
fig.savefig(f"overview_methods_line_grid.svg")
fig.savefig(f"overview_methods_line_grid.pdf")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True, sharey=True)
sns.scatterplot(
    x=df_sm.index,
    y="Saliency Map",
    data=df_sm,
    ax=axes[0, 0],
    color=sns.color_palette()[0],
)
sns.scatterplot(
    y="GradCam",
    x=df_gradcam.index,
    data=df_gradcam,
    ax=axes[0, 1],
    color=sns.color_palette()[1],
)
sns.scatterplot(
    y="GradCam Cutted",
    x=df_gradcam_cutted.index,
    data=df_gradcam_cutted,
    ax=axes[1, 0],
    color=sns.color_palette()[2],
)
sns.scatterplot(
    y="GradCam++",
    x=df_gcpp.index,
    data=df_gcpp,
    ax=axes[1, 1],
    color=sns.color_palette()[3],
)
axes[0, 0].set_title("Saliency Map")
axes[0, 1].set_title("GradCam")
axes[1, 0].set_title("GradCam Cutted")
axes[1, 1].set_title("GradCam++")
#  Make common axis labels
axes[-1, 0].set_ylabel(".", color=(0, 0, 0, 0))
axes[-0, 0].set_ylabel(".", color=(0, 0, 0, 0))
fig.text(0.5, 0.04, "Test Accuracy", va="center", ha="center")
fig.text(
    0.04, 0.5, "Anzahl maskierte Zellen", va="center", ha="center", rotation="vertical"
)

fig.savefig(f"overview_methods_scatter_grid.png")
fig.savefig(f"overview_methods_scatter_grid.svg")
fig.savefig(f"overview_methods_scatter_grid.pdf")

# Stock Images

## Original

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Original Stock" \
  --datasets $sticker_org_path \
  --mean $mean \
  --std $std \

## Manipulated

### Fortuna

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path  \
  --method_names "Fortuna Large" \
  --datasets $sticker_f_large_path \
  --mean $mean \
  --std $std

! mv /content/large_sign.png /content/fortuna_large_sign.png
! mv /content/large_sign.pdf /content/fortuna_large_sign.pdf
! mv /content/large_sign.svg /content/fortuna_large_sign.svg
! mv /content/large_sign_df.csv /content/fortuna_large_sign_df.csv

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path  \
  --method_names "Fortuna Medium" \
  --datasets $sticker_f_medium_path \
  --mean $mean \
  --std $std

! mv /content/medium_sign.png /content/fortuna_medium_sign.png
! mv /content/medium_sign.pdf /content/fortuna_medium_sign.pdf
! mv /content/medium_sign.svg /content/fortuna_medium_sign.svg
! mv /content/medium_sign_df.csv /content/fortuna_medium_sign_df.csv

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path  \
  --method_names "Fortuna Small" \
  --datasets $sticker_f_small_path \
  --mean $mean \
  --std $std

! mv /content/small_sign.png /content/fortuna_small_sign.png
! mv /content/small_sign.pdf /content/fortuna_small_sign.pdf
! mv /content/small_sign.svg /content/fortuna_small_sign.svg
! mv /content/small_sign_df.csv /content/fortuna_small_sign_df.csv

### Bochum

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Bochum Large" \
  --datasets $sticker_b_large_path \
  --mean $mean \
  --std $std

! mv /content/large_sign.png /content/bochum_large_sign.png
! mv /content/large_sign.pdf /content/bochum_large_sign.pdf
! mv /content/large_sign.svg /content/bochum_large_sign.svg
! mv /content/large_sign_df.csv /content/bochum_large_sign_df.csv

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Bochum Medium" \
  --datasets $sticker_b_medium_path \
  --mean $mean \
  --std $std

! mv /content/medium_sign.png /content/bochum_medium_sign.png
! mv /content/medium_sign.pdf /content/bochum_medium_sign.pdf
! mv /content/medium_sign.svg /content/bochum_medium_sign.svg
! mv /content/medium_sign_df.csv /content/bochum_medium_sign_df.csv

In [ ]:
! python /content/GTSRB_Visualization/code/test_performance.py \
  --checkpoint $trained_model_path \
  --method_names "Bochum Small" \
  --datasets $sticker_b_small_path \
  --mean $mean \
  --std $std

! mv /content/small_sign.png /content/bochum_small_sign.png
! mv /content/small_sign.pdf /content/bochum_small_sign.pdf
! mv /content/small_sign.svg /content/bochum_small_sign.svg
! mv /content/small_sign_df.csv /content/bochum_small_sign_df.csv

### Merge Results

In [ ]:
org_df = create_clean_df(
    base_path.joinpath("original_df.csv"), ("Test accuracy", "Fortuna"), "Original Stock"
)
df_fortuna_large = create_clean_df(
    base_path.joinpath("fortuna_large_sign_df.csv"), ("Test accuracy", "Fortuna"), "Fortuna Large"
)
df_fortuna_medium = create_clean_df(
    base_path.joinpath("fortuna_medium_sign_df.csv"),
    ("Test accuracy", "Fortuna"),
    "Fortuna Medium",
)
df_fortuna_small = create_clean_df(
    base_path.joinpath("fortuna_small_sign_df.csv"), ("Test accuracy", "Fortuna"), "Fortuna Small"
)
df_fortuna = pd.concat(
    [org_df, df_fortuna_large, df_fortuna_medium, df_fortuna_small], ignore_index=True
)
df_fortuna.rename(
    index={0: "Original", 1: "Large", 2: "Medium", 3: "Small"}, inplace=True
)

In [ ]:
org_df = create_clean_df(
    base_path.joinpath("original_df.csv"), ("Test accuracy", "Bochum"), "Original Stock"
)
df_bochum_large = create_clean_df(
    base_path.joinpath("bochum_large_sign_df.csv"), ("Test accuracy", "Bochum"), "Bochum Large"
)
df_bochum_medium = create_clean_df(
    base_path.joinpath("bochum_medium_sign_df.csv"), ("Test accuracy", "Bochum"), "Bochum Medium"
)
df_bochum_small = create_clean_df(
    base_path.joinpath("bochum_small_sign_df.csv"), ("Test accuracy", "Bochum"), "Bochum Small"
)
df_bochum = pd.concat(
    [org_df, df_bochum_large, df_bochum_medium, df_bochum_small], ignore_index=True
)
df_bochum.rename(
    index={0: "Original", 1: "Large", 2: "Medium", 3: "Small"}, inplace=True
)

In [ ]:
result_sticker = pd.concat([df_fortuna, df_bochum], axis=1, sort=False)
result_sticker.to_csv("overview_sticker.csv")
result_sticker

In [ ]:
sns.set_theme(style="whitegrid")
splot = sns.lineplot(data=result_sticker, markers=True)
splot.set(xlabel="Schilder Größe", ylabel="Test Accuracy")
splot.xaxis.set_major_locator(ticker.MultipleLocator(1))
splot.xaxis.set_major_formatter(ticker.ScalarFormatter())
splot.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
splot.yaxis.set_major_formatter(ticker.ScalarFormatter())
splot.set_xticks(range(len(result_sticker)))
splot.set_xticklabels(["None", "Groß", "Mittel", "Klein"])
splot.figure.savefig(f"overview_sticker_lines.png")
splot.figure.savefig(f"overview_sticker_lines.svg")
splot.figure.savefig(f"overview_sticker_lines.pdf")

In [ ]:
sns.set_theme(style="whitegrid")
splot = sns.scatterplot(data=result_sticker, markers=True)
splot.set(xlabel="Schilder Größe", ylabel="Test Accuracy")
splot.xaxis.set_major_locator(ticker.MultipleLocator(1))
splot.xaxis.set_major_formatter(ticker.ScalarFormatter())
splot.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
splot.yaxis.set_major_formatter(ticker.ScalarFormatter())
splot.set_xticks(range(len(result_sticker)))
splot.set_xticklabels(["None", "Groß", "Mittel", "Klein"])
splot.figure.savefig(f"overview_sticker_scatter.png")
splot.figure.savefig(f"overview_sticker_scatter.svg")
splot.figure.savefig(f"overview_sticker_scatter.pdf")

# Real Examples

In [ ]:
mean = mean_list
std = std_list

In [ ]:
def test_image(img_path):
    image = Image.open(img_path).convert("RGB")
    tensor = preprocess(image)
    tensor = tensor.unsqueeze(0)
    y_pred = model(tensor)
    sm = torch.nn.Softmax()
    a, y_hat = torch.max(sm(y_pred), dim=1)
    return a, y_hat

In [ ]:
preprocess = transforms.Compose(
    [
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

model = LitModel.load_from_checkpoint(
    trained_model_path,
    mean=mean,
    std=std,
    train_dataset=None,
    test_dataset=None,
)
model.eval()

tuples = [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]
targets = [
    "00012",
    "00012",
    "00012",
    "00012",
    "00027",
    "00027",
    "00038",
    "00038",
    "00039",
    "00039",
]
examples = sorted([x for x in real_examples_path.rglob("*.png")])

fig, axes = plt.subplots(5, 2, figsize=(20, 20))
ax = axes.ravel()

for t in tuples:
    a_org, y_hat_org = test_image(str(examples[t[0]]))
    a_fixed, y_hat_fixed = test_image(str(examples[t[1]]))
    ax[t[0]].imshow(io.imread(examples[t[0]]))
    ax[t[0]].set_xlabel(
        "Wahre Klasse: "
        + r"$\bf{}$".format(targets[t[0]])
        + "\nKlassifizierung "
        + r"$\bf{}$".format(str(y_hat_org[0].item()).zfill(5))
        + " mit \n einer Wahrscheinlichkeit von "
        + r"$\bf{:10.4f}$".format(a_org[0]),
        horizontalalignment="center",
    )
    ax[t[0]].set_xticks([])
    ax[t[0]].set_yticks([])
    ax[t[1]].imshow(io.imread(examples[t[1]]))
    ax[t[1]].set_xlabel(
        "Wahre Klasse: "
        + r"$\bf{}$".format(targets[t[1]])
        + "\nKlassifizierung "
        + r"$\bf{}$".format(str(y_hat_fixed[0].item()).zfill(5))
        + " with \n a probability of "
        + r"$\bf{:10.4f}$".format(a_fixed[0]),
        horizontalalignment="center",
    )
    ax[t[1]].set_xticks([])
    ax[t[1]].set_yticks([])


fig.tight_layout()
plt.show()
fig.savefig(f"real_examples.png")
fig.savefig(f"real_examples.svg")
fig.savefig(f"real_examples.pdf")

# Download created files
Download the created files. Not necessary if you use a local system.

In [ ]:
p = Path(r"/content/").glob("./*")
files = [x for x in p if x.is_file()]

In [ ]:
suffix_list = [".pdf", ".png", ".svg", ".csv"]

with zipfile.ZipFile("result.zip", "w") as zipF:
    for file in files:
        if file.suffix in suffix_list:
            zipF.write(str(file), compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
colab.files.download("/content/result.zip")